In [2]:
path = r"D:\datasets\mimiciii"

In [3]:
import pandas as pd
import vaex
import os
import datetime
import re
import time
os.chdir(path)

In [4]:
%%time
chartevents = vaex.open("CHARTEVENTS.csv.hdf5")
labevents = vaex.open("LABEVENTS.csv.hdf5")
inputevents_cv = vaex.open("INPUTEVENTS_CV.csv.hdf5")
inputevents_mv = vaex.open("INPUTEVENTS_MV.csv.hdf5")
predcriptions = vaex.open("PRESCRIPTIONS.csv.hdf5")

Wall time: 152 ms


In [5]:
%%time
icustays = pd.read_csv("ICUSTAYS.csv",index_col=0,parse_dates=[9,10])
procedure_mv = pd.read_csv("PROCEDUREEVENTS_MV.csv",index_col=0,parse_dates=[4,5])
procedure_icd = pd.read_csv("PROCEDURES_ICD.csv")

Wall time: 3.81 s


In [6]:
os.chdir(r"F:\myresearch\TraumaHba1\data2")

In [7]:
admissions = pd.read_csv("trauma_admissions.csv",index_col=0,parse_dates=[3,4,5,14,15,21,22,23,24])

In [8]:
icu_trauma = admissions[admissions.HADM_ID.isin(icustays.HADM_ID)]

In [9]:
icu_hadm_id = icu_trauma.HADM_ID

In [10]:
labevents_trauma = labevents[labevents.HADM_ID.isin(icu_hadm_id)]

chartevents_trauma = chartevents[chartevents.HADM_ID.isin(icu_hadm_id)]

predcriptions_trauma = predcriptions[predcriptions.HADM_ID.isin(icu_hadm_id)]

procedure_icd_trauma = procedure_icd[procedure_icd.HADM_ID.isin(icu_hadm_id)]

procedure_mv_trauma = procedure_mv[procedure_mv.HADM_ID.isin(icu_hadm_id)]

In [23]:
"""
从labevents 表中提取首次指标，加入总表
lab_item_id:字符串，所需项目的ITEMID,为单一值
variable_name:目标表格中新变量(新列)的名称
注意事项：1、总表需要定义成全局变量，并且需要根据具体情况改名
          2、仅适用vaex DataFrame格式数据表
"""
def add_vari_labevents_id_hdf(lab_item_id,variable_name):
    
    global icu_trauma
     
    items = labevents_trauma[labevents_trauma["ITEMID"]==lab_item_id]
    
    items_df = items.to_pandas_df()

    # bpd_chart_df.info()

    items_df["CHARTTIME"] = items_df["CHARTTIME"].apply(lambda x:pd.to_datetime(x))
   
    first_time = items_df.groupby("HADM_ID").agg({"CHARTTIME":"min"})

    events=pd.merge(first_time,items_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

    events.drop_duplicates(subset=["HADM_ID"],inplace=True)
    
    icu_trauma = pd.merge(icu_trauma,events,on="HADM_ID",how="left")

    icu_trauma.rename(columns={"VALUENUM":variable_name},inplace=True)

In [24]:
labitems = pd.read_excel("labitems_id.xlsx")

In [25]:
%%time
for itemid,lab_item in zip(labitems.itemid,labitems.lab_items):
    #print(itemid)
    #print(lab_item)
    add_vari_labevents_id_hdf(itemid,lab_item)
    print(lab_item+" has been added.")

SPECIMEN has been added.
AADO2(art) has been added.
BASEEXCESS(art) has been added.
BICARBONATE(art) has been added.
TOTALCO2(art) has been added.
CARBOXYHEMOGLOBIN(art) has been added.
CHLORIDE(art) has been added.
CALCIUM(art) has been added.
GLUCOSE(art) has been added.
HEMATOCRIT(art) has been added.
HEMOGLOBIN(art) has been added.
INTUBATED has been added.
LACTATE(art) has been added.
METHEMOGLOBIN(art) has been added.
O2FLOW has been added.
FIO2 has been added.
SO2(art) has been added.
PCO2(art) has been added.
PEEP has been added.
PH(art) has been added.
PO2(art) has been added.
POTASSIUM(art) has been added.
REQUIREDO2 has been added.
SODIUM(art) has been added.
TEMPERATURE has been added.
TIDALVOLUME has been added.
VENTILATIONRATE has been added.
VENTILATOR has been added.
HbA1c% has been added.
HbA1c has been added.
ALT has been added.
ALBUMIN has been added.
Amylase has been added.
ANIONGAP has been added.
AST has been added.
BICARBONATE has been added.
DB has been added.
I

In [21]:
"""
从chartevents 表中提取首次指标，加入总表
chart_id_list:列表，所需项目的ITEMID集合
variable_name:目标表格中新变量(新列)的名称
注意事项：1、总表需要定义成全局变量，并且需要根据具体情况改名
          2、仅适用vaex DataFrame格式数据表
"""
def add_vari_chartevents(chart_id_list,variable_name):
    
    global icu_trauma
     
    chart = chartevents_trauma[chartevents_trauma["ITEMID"].isin(chart_id_list)]
    
    chart_df = chart.to_pandas_df()

    # bpd_chart_df.info()

    chart_df[["CHARTTIME","STORETIME"]] = chart_df[["CHARTTIME","STORETIME"]].apply(lambda x:pd.to_datetime(x))
   
    first_time = chart_df.groupby("HADM_ID").agg({"CHARTTIME":"min"})

    events=pd.merge(first_time,chart_df,on="CHARTTIME",how="left")[["HADM_ID","VALUENUM"]]

    events.drop_duplicates(subset=["HADM_ID"],inplace=True)
    
    icu_trauma = pd.merge(icu_trauma,events,on="HADM_ID",how="left")

    icu_trauma.rename(columns={"VALUENUM":variable_name},inplace=True)

In [11]:
chartitems = pd.read_excel("chartitems_id.xlsx")

In [12]:
dict(zip(chartitems.chart_items,chartitems.itemid))

{'HeartRate': '[211,220045]',
 'SysBP': '[51,442,455,6701,220179,220050]',
 'DiasBP': '[8368,8440,8441,8555,220180,220051]',
 'MeanBP': '[456,52,6702,443,220052,220181,225312]',
 'RespRate': '[615,618,220210,224690]',
 'TempF': '[223761,678]',
 'TempC': '[223762,676]',
 'SpO2': '[646,220277]',
 'Glucose': '[807,811,1529,3745,3744,225664,220621,226537]'}

In [13]:
chartitems_dict = {'HeartRate': [211,220045],
                   'SysBP': [51,442,455,6701,220179,220050],
                   'DiasBP': [8368,8440,8441,8555,220180,220051],
                   'MeanBP': [456,52,6702,443,220052,220181,225312],
                   'RespRate': [615,618,220210,224690],
                   'TempF': [223761,678],
                   'TempC': [223762,676],
                   'SpO2': [646,220277],
                   'Glucose': [807,811,1529,3745,3744,225664,220621,226537]
                   }

In [22]:
%%time
for chart_item,itemid in chartitems_dict.items():
    #print(itemid)
    #print(chart_item)
    add_vari_chartevents(itemid,chart_item)
    print(chart_item+" has been added.")

HeartRate has been added.
SysBP has been added.
DiasBP has been added.
MeanBP has been added.
RespRate has been added.
TempF has been added.
TempC has been added.
SpO2 has been added.
Glucose has been added.
Wall time: 39min 26s


In [26]:
icu_trauma.shape

(1032, 132)

In [27]:
icu_trauma.to_csv("icu_trauma.csv",index=0)